<a href="https://colab.research.google.com/github/UAPH451551/PH451_551_Sp23/blob/main/Resources/Synthetic_Data_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import imblearn
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
california = datasets.fetch_california_housing()

In [ ]:
data, target = california['data'], california['target']

In [ ]:
plt.hist(data[:,0])
plt.show()

In [ ]:
under_rep_class = data[data[:,0] > 12]
over_rep_class = data[data[:,0] < 12]
under_rep_labels = np.zeros(len(under_rep_class))
over_rep_labels = np.ones(len(over_rep_class))

In [ ]:
all_classes = np.concatenate((over_rep_class, under_rep_class))
all_labels = np.concatenate((over_rep_labels, under_rep_labels))

In [ ]:
plt.hist(over_rep_class[:,0], range=(0,16))
plt.hist(under_rep_class[:,0], range=(0,16))
plt.show()

In [ ]:
plt.hist(under_rep_class[:,0], range=(12,16))
plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
oversample = SMOTE()
all_classes_2, all_labels_2 = oversample.fit_resample(all_classes, all_labels)

In [ ]:
plt.hist(all_classes_2[:,0])

In [ ]:
new_under_rep = all_classes_2[all_classes_2[:,0] > 11]

In [ ]:
plt.hist(under_rep_class[:,0], histtype='step', density=1, color='blue', label='Old')
plt.hist(new_under_rep[:,0], histtype='step', density=1, color='orange', label='New')
plt.legend()
plt.title('Median Income')
plt.show()

In [ ]:
plt.hist(under_rep_class[:,2], histtype='step', density=1, color='blue', label='Old')
plt.hist(new_under_rep[:,2], histtype='step', density=1, color='orange', label='New')
plt.title('Average Rooms')
plt.legend()
plt.show()

In [ ]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv

In [ ]:
import pandas as pd

In [ ]:
temp_data = pd.read_csv('daily-min-temperatures.csv')

In [ ]:
print(temp_data)

In [ ]:
time_series = temp_data['Temp']

In [ ]:
plt.plot(time_series)
plt.show()

In [ ]:
steps = np.arange(len(time_series))

In [ ]:
from scipy.interpolate import CubicSpline

In [ ]:
cs = CubicSpline(steps, time_series)

In [ ]:
print(cs.c)

In [ ]:
plt.plot(steps, cs(steps))
new_steps = np.arange(len(steps), len(steps)+2)
plt.plot(new_steps, cs(new_steps))
plt.show()

In [ ]:
smooth_series = time_series.rolling(60).mean()

In [ ]:
c_smooth_series = smooth_series.tail(smooth_series.shape[0] - 60)
c_steps = steps[60:]

In [ ]:
print(len(c_steps))

In [ ]:
plt.plot(c_smooth_series)
plt.show()

In [ ]:
import scipy.optimize
#https://stackoverflow.com/questions/16716302/how-do-i-fit-a-sine-curve-to-my-data-with-pylab-and-numpy
def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", 
    "omega", "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = np.array(tt)
    yy = np.array(yy)
    ff = np.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(np.fft.fft(yy))
    guess_freq = abs(ff[np.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = np.std(yy) * 2.**0.5
    guess_offset = np.mean(yy)
    guess = np.array([guess_amp, 2.*np.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * np.sin(w*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*np.pi)
    fitfunc = lambda t: A * np.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, 
            "fitfunc": fitfunc, "maxcov": np.max(pcov), "rawres": (guess,popt,pcov)}

In [ ]:
res = fit_sin(c_steps, c_smooth_series)

In [ ]:
fitted_series = res["fitfunc"](c_steps)

In [ ]:
plt.plot(c_steps, fitted_series)
plt.show()

In [ ]:
plt.plot(time_series[60:] - fitted_series)
plt.show()

In [ ]:
residuals = time_series[60:] - fitted_series

In [ ]:
smooth_residuals = residuals.rolling(60).mean()

In [ ]:
c_smooth_residuals = smooth_residuals.tail(smooth_residuals.shape[0] - 60)
c_steps_2 = c_steps[60:]

In [ ]:
res2 = fit_sin(c_steps_2, c_smooth_residuals)

In [ ]:
plt.plot(c_steps_2, c_smooth_residuals)
plt.show()

In [ ]:
residuals_fitted_series = res2["fitfunc"](c_steps_2)

In [ ]:
plt.plot(c_steps_2, residuals_fitted_series)
plt.show()

In [ ]:
plt.plot(time_series[120:] - fitted_series[60:] - residuals_fitted_series)
plt.show()

In [ ]:
gaussian_noise = time_series[120:] - fitted_series[60:] - residuals_fitted_series

In [ ]:
plt.hist(gaussian_noise)
plt.show()

In [ ]:
gaussian_mean = np.mean(gaussian_noise)
gaussian_std = np.std(gaussian_noise)

In [ ]:
new_gaussian = np.random.normal(loc=gaussian_mean, scale=gaussian_std, size=len(gaussian_noise))

In [ ]:
plt.hist(gaussian_noise, histtype='step', color='blue')
plt.hist(new_gaussian, histtype='step', color='orange')
plt.show()

In [ ]:
plt.plot(steps, time_series, alpha=.5)
plt.plot(c_steps_2, fitted_series[60:] + residuals_fitted_series + new_gaussian, alpha=.5)
plt.show()